# 1 - Swiyu Signup and Exploration

Linus: Add Taxonomy overview with Legal / Trust and Threats / Cryptography.

Short overview of what we'll be doing:

1. Installing and creating credential
2. Using the HoW credential
3. Inspect messages from logs
4. Check linkability of presentations

## 1.1 - Install the SWIYU app

The SWIYU project includes a wallet app that we use to store the credentials obtained from the the different issuers that idhere to the SWIYU issuer specification.  
The wallet will also be used to perform and provide verifications to verifiers that adhere to the SWIYU verifiers specification

a.	https://play.google.com/store/apps/details?id=ch.admin.foitt.swiyu&hl=en&pli=1  
b.	https://apps.apple.com/ch/app/swiyu/id6737259614

## 1.2 - Get a Beta Government E-ID

from: https://www.bcs.admin.ch/bcs-web/#/beta-id/create

## 1.3 - Create an E-ID HoW Credential

Go to https://eid.c4dt.org and navigate to the issuer section  
a.	The website will ask you to verify you identity first  
b.	then it will issue you a credential for attending the workshop  
c.  All data will be populated from the Beta ID credential you verified except the grade, you will get to set that yourself.  
d.	You will receive the credential on your wallet.

## 1.4 - Verify your Credential

Go to https://eid.c4dt.org/verifier/verify where you will verify your newly gained VC.

## 1.5 - Inspect Logs

Go to https://eid.c4dt.org/logs where you will find the server logs that match your queries.  
Look  for the logs that match your VC presentation (the request that goes out from your wallet when you scan the QR code to verify your newly acquired VC.)

## 1.6 - Analyze Logs
1.	Find your specific VC presentation request in the logs. Hint: look for the:
    a.	Timestamp - when you made the request  
    b.	The request content is preceeded by `=== REQUEST BODY ===`
    c.  And the body is preceeded by `=== RESPONSE BODY ===`
2.	The presentation is URL-encoded, so decode it first. Use any online tool 
    i.	You might still want to put in a better format.. an LLM might be able to help (not necessary).
3.	From the decoded request body, find the encoded VC presentation. 
4.	Use https://www.sdjwt.co/decode to decode the presentation.
5.  Identify the different types of information stored there.
    Which parts do you think can be used to recog

##### 1.7 - Linkability of Multiple Presentations

Do two verifications of the HoW credential, fetch the presentation from the Logs, and search for fixed strings.

Discussion:

- What can make it very hard to avoid linkability when you think about the fields you're sharing with the Verifier?
- Which fields can be revealed and keep anonymity?
- Check with your neighbours to see which fields are constant between two wallets
- What is a simple way to remove some of the linkability?